In [1]:
import pandas as pd
import string
import re
from nltk.stem.porter import *
stemmer = PorterStemmer()
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import ToktokTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [2]:
df1 = pd.read_csv("pofmem_examples.csv",encoding='latin')
df1.columns

Index(['id', 'host_about'], dtype='object')

# Text Cleansing

In [3]:
df1['host_about'] = df1.host_about.astype(str)
lines = df1['host_about'].values.tolist()
df1.head()

,id,host_about
0,0.0,I would describe myself as a very determined a...
1,0.0,T&A Real Estate has adopted the principle of g...
2,0.0,Hello We live in Soho and love it here - Brit...
3,0.0,"A lover of the outdoors. I enjoy kayaking, hik..."
4,0.0,Hello everyone and an advance welcome to Hong ...


In [4]:
# function to clean text
def review_to_words(txt):
    cleaned_txt = re.sub("[^a-zA-Z\.\!\?\n]", " ", txt)
    words = cleaned_txt.lower().split()
    return( " ".join( words ))

In [5]:
df1['clean_description'] = [review_to_words(text) for text in df1['host_about']]
df1.head()

,id,host_about,clean_description
0,0.0,I would describe myself as a very determined a...,i would describe myself as a very determined a...
1,0.0,T&A Real Estate has adopted the principle of g...,t a real estate has adopted the principle of g...
2,0.0,Hello We live in Soho and love it here - Brit...,hello we live in soho and love it here british...
3,0.0,"A lover of the outdoors. I enjoy kayaking, hik...",a lover of the outdoors. i enjoy kayaking hiki...
4,0.0,Hello everyone and an advance welcome to Hong ...,hello everyone and an advance welcome to hong ...


In [6]:
# plot_model(lda, plot='topic_model')

# PofMeM

In [7]:
from transformers import pipeline
#### siebert/sentiment-roberta-large-english
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

#### cardiffnlp/twitter-xlm-roberta-base-sentiment
# model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
# sentiment_analysis = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

Some weights of RobertaModel were not initialized from the model checkpoint at siebert/sentiment-roberta-large-english and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
doc = df1['clean_description']
doc.head()

0    i would describe myself as a very determined a...
1    t a real estate has adopted the principle of g...
2    hello we live in soho and love it here british...
3    a lover of the outdoors. i enjoy kayaking hiki...
4    hello everyone and an advance welcome to hong ...
Name: clean_description, dtype: object

In [9]:
topic_dict={'Topic1':{'Words':{"people","meet","world","culture","different","enjoy","meeting","place","live","experience","explore","host","work","learn","country","currently","life","share","food","interesting","passion","discover","always","see","professional","look","adventure","traveler","passionate","medium"}}
,'Topic2':{'Words':{"happy","help","always","make","good","need","know","guest","try","give","tidy","thing","well","chat","host","time","recommendation","privacy","friendly","keep","work","let","may","also","get","talk","want","day","enjoyable","ask"}}
,'Topic3':{'Words':{"restaurant","walk","area","room","apartment","cafe","close","bar","minute","shop","local","away","offer","also","park","coffee","location","beach","beautiful","quiet","great","space","private","house","access","comfort","living","night","locate","neighborhood"}}
,'Topic4':{'Words':{"friendly","easy","go","person","clean","respectful","people","life","guy","treat","quiet","fun","professional","outgoing","respect","would","like","enjoy","responsible","keep","couple","lifestyle","active","healthy","kind","care","happy","nice","young","always"}}
,'Deontology':{'Words':{"duty","duties","responsibilit","banned","taboo","allowed","calling","authoriz","permitted","commitment","contract","dues","mission","obligat","role","task","tasks","accountab","requirement","rights","required","necessary","compulsory","demanded","mandatory","vital","obey","adher","comply","complies","complied","complying","disobey","forbidden","prohibited","code","codes","transgress","sanctioned","acceptable","violat","principle","standards","rule","rules","command","fundamental","guideline","law","maxim","proscription","proscribe","prescription","prescribe","regulation","ruling","tenet","etiquette","convention","custom","norm","practice","doctrine"}}
,'Consequentialism':{'Words':{"result","profit","outcome","consequen","effects","effect","repercussion","prosper","utilit","welfare","wealth","suffer","pain","painful","pains","pained","happiness","health","interests","flourish","success","successes","pleasure","well-being","benefi","advantage","maximi","gain","loss"}}
}


In [10]:
Topiclist = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Deontology', 'Consequentialism']

In [11]:
def pofMeM(document):
    df_FM = pd.DataFrame(columns=Topiclist)
    df_TS = pd.DataFrame(columns=Topiclist)
    df_TS_freq_pos  = pd.DataFrame(columns=Topiclist)
    df_TS_freq_neg  = pd.DataFrame(columns=Topiclist)
    for i in range(len(document)):
        try:
            f = document[i]
            lines =[]
            lines = re.split('\.\s|\?|\!',f)
            fs = {}
            ts = {}
            tsp= {}
            tsn= {}
            for tp in Topiclist:
                words = topic_dict[tp]['Words']
                total = 0
                count = defaultdict(lambda: 0)
                for word in re.sub(r'[^\w\s]', '', f).split(' '):
                    total += 1
                    if any(keyword in word for keyword in words):
                        count[tp] += 1

                topic_include_sentences = []
                pos= 0
                neg= 0
                for each_line in lines:
                    try:
                        if any(keyword in each_line for keyword in words):
                            topic_include_sentences.append(each_line)
                            if topic_include_sentences !=[]:
                                topic_include_sentences = topic_include_sentences[:512]
                                if(sentiment_analysis(each_line)[0]['label'].lower()=="negative"): neg += 1
                                else:pos += 1
                        else:
                            pass
                    except Exception as ex:
                        print(ex)   
                if topic_include_sentences ==[]:
                    fs[tp] = None
                    ts[tp] = None
                    tsp[tp] = pos
                    tsn[tp] = neg
                else:
                    sent = sentiment_analysis(". ".join(topic_include_sentences))
                    if(sent[0]['label'].lower()=="negative"): value = sent[0]['score']*-1
                    else:value = sent[0]['score']*1
                    ts[tp] = value
                    tsp[tp] = pos
                    tsn[tp] = neg
                    fs[tp] = count[tp]*100*value/total
                    print(f'Feature:[{tp}]\nScore: [{round(fs[tp],3)}]\n #of positive sentences:{pos}\n #of negative sentences:{neg} \nSentences:{topic_include_sentences}\n\n')
            df_FM = df_FM.append(fs, ignore_index = True)
            df_TS = df_TS.append(ts, ignore_index = True)
            df_TS_freq_pos = df_TS_freq_pos.append(tsp, ignore_index = True)
            df_TS_freq_neg = df_TS_freq_neg.append(tsn, ignore_index = True)
            print("\r", "Progress {:2.4%}".format((i+1) / len(document)), end="" + '\n\n\n\n')
        except Exception as ex:
            print(ex)   
    return df_FM, df_TS, df_TS_freq_pos, df_TS_freq_neg

In [12]:
FM, TS, FP, FN = pofMeM(doc)

Feature:[Topic1]
Score: [3.43]
 #of positive sentences:5
 #of negative sentences:0 
Sentences:['i take my job seriously but able to see things in perspective and i m very easy going to work with', 'i would also like to add that your stay and experience is of the utmost importance to me therefore i am available before the booking and during the stay to answer all your concerns and anxieties', 'for example where one would go to eat and taste the local culinary delights easy and cheap ways to travel around london this can be a daunting experience ', 'linens are provided during your stay and there is access to a fully working kitchen and laundry facilities', 'and working with the public is certainly something i look forward to and enjoy.']


Feature:[Topic2]
Score: [4.716]
 #of positive sentences:6
 #of negative sentences:0 
Sentences:['i take my job seriously but able to see things in perspective and i m very easy going to work with', 'i am an optimist rather than pessimist and cope well 

Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


index out of range in self
Feature:[Topic1]
Score: [2.219]
 #of positive sentences:6
 #of negative sentences:0 
Sentences:['i enjoy kayaking hiking and people watch yep', 'i ve traveled throughout europe and lived in the caribbean for several years', 'i spend most of my free time working with at risk youth serve as a mentor to black and hispanic high school girls', 'and because i cannot get enough i volunteer my time at non profit organizations that invest in preparing women who have been recently incarcerated with the necessary skills that will help them to reintegrate in the work place and in society', 'security cameras are located in shared spaces throughout the building for residents safety', 'always available via text or call.']


Feature:[Topic2]
Score: [3.878]
 #of positive sentences:5
 #of negative sentences:4 
Sentences:[' that s a thing ', 'with that said i do not welcome guests with dogs or any type of animals in my home', 'i spend most of my free time working with at risk y